## 1.1 
- 
1

In [60]:
from pyspark.rdd import RDD
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
import pyspark
from pyspark.sql import functions as sf

In [61]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "ASSIGNMENT 1"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkSession classes 
from pyspark.sql import SparkSession # Spark SQL

from pyspark import SparkContext # Spark
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')


- 2

In [62]:
#Importing all 'Units' csv files into a single RDD object named as rd_units
rd_units = sc.textFile('*_Units.csv')

- 3

In [63]:
#Importing all 'Crash' csv files into a single RDD object named as rd_crash
rd_crash = sc.textFile('*_Crash.csv')

In [64]:
#Cleaning rdds
rd_units = rd_units.map(lambda x : (x.replace('"','')))
rd_crash = rd_crash.map(lambda x : (x.replace('"','')))

- 4

In [65]:
#Removing header row for the rdd rd_units
rd_units = rd_units.map(lambda line_units : line_units.split(','))
header_units = rd_units.first()
rd_units = rd_units.filter(lambda row_units : row_units != header_units )

#Removing header row for the rdd rd_crash
rd_crash = rd_crash.map(lambda line_crash : line_crash.split(','))
header_crash = rd_crash.first()
rd_crash = rd_crash.filter(lambda row_crash : row_crash != header_crash )
 
#Printing count of rows in rdd rd_units
print("Count of records in rdd - rd_units is " + str(rd_units.count()) + "\n")

#Printing count of rows in rdd rd_crash
print("Count of records in rdd - rd_crash is " + str(rd_crash.count()) + "\n")

#Printing top 10 rows in rdd rd_units
print("Top 10 records of rdd - rd_units is " + str(rd_units.take(10)) + "\n")

#Printing top 10 rows in rdd rd_crash
print("Top 10 records of rdd - rd_crash is " + str(rd_crash.take(10)))

Count of records in rdd - rd_units is 153854

Count of records in rdd - rd_crash is 72006

Top 10 records of rdd - rd_units is [['2016-1-15/08/2019', '01', '0', 'SA', 'OMNIBUS', '2011', 'North', 'Male', '056', 'SA', 'HR', 'Full', 'Not Towing', 'Straight Ahead', '010', '5121', '', ''], ['2016-1-15/08/2019', '02', '1', '', 'Pedestrian on Road', '', 'East', 'Male', '072', '', '', '', '', 'Walking on Road', '', '5084', '', ''], ['2016-2-15/08/2019', '01', '0', 'SA', 'Motor Cars - Sedan', '2004', 'Unknown', 'Female', '023', 'SA', 'C ', 'Full', 'Not Towing', 'Straight Ahead', '001', '5087', '', ''], ['2016-2-15/08/2019', '02', '0', 'SA', 'Station Wagon', '2008', 'Unknown', 'Male', '040', 'SA', 'C ', 'Full', 'Not Towing', 'Straight Ahead', '001', '5084', '', ''], ['2016-3-15/08/2019', '01', '0', 'SA', 'RIGID TRUCK LGE GE 4.5T', '1990', 'South', 'Unknown', 'XXX', 'SA', 'MR', 'Provisional 2', 'Not Towing', 'Straight Ahead', '001', '5115', '', ''], ['2016-3-15/08/2019', '02', '0', 'SA', 'Panel V

## 1.2

- 1

In [66]:
#Printing number of paartitions in each rdd
print("Number of partitions in rdd - rd_units is " + str(rd_units.getNumPartitions()) + "\n")
print("Number of partitions in rdd - rd_crash is " + str(rd_crash.getNumPartitions()))

Number of partitions in rdd - rd_units is 5

Number of partitions in rdd - rd_crash is 5


*  ### The number of partitions in the above two rdds i.e. rd_units and rd_crash are 5 each. This can be seen from using the function .getNumPartitions() ###

*  ### When we do not define the way of partitioning in an rdd, the rdd object itself by default does Round Robin equal partitioning. In this approach the rdd is divided into equal chunks of data and given to the different processing units. This way no processor sits idle while another processor is working on some task.  ###


- 2

- a

In [67]:
#function to parse a record in the rdd
def parsingARecord(line):
    array_line = line[0:9] + line[10:]
    # Return a tuple with the lic state as first element and the remaining as the second element
    return (line[9], array_line)

rd_units_pair = rd_units.map(parsingARecord)


 - b

In [68]:
no_of_partitions = 2

#hash function to generate key value pairs
def hash_function(key):
    total = 0
    if key == 'SA':
        total = 0
    else:
        total = 1
    return total


rd_units_partitioned = rd_units_pair.partitionBy(no_of_partitions, hash_function)

- c

In [69]:
#A Function to print the data items in each RDD
def print__the_partitions(data):
    if isinstance(data, RDD):
        numPartitions = data.getNumPartitions()
        partitions = data.glom().collect()
    else:
        numPartitions = data.rdd.getNumPartitions()
        partitions = data.rdd.glom().collect()
    
    print(f"####### NUMBER OF PARTITIONS: {numPartitions}")
    for index, partition in enumerate(partitions):
        # show partition if it is not empty
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records")


print__the_partitions(rd_units_partitioned)            


####### NUMBER OF PARTITIONS: 2
Partition 0: 109684 records
Partition 1: 44170 records


* ### The above code displays the number of records in each partition. We can see that Partition 0 has 109,684 records and Partition 1 has 44,170 records. ###



* ### As a result there is data skewness present as can be seen from the count of each partition. Partition 0(with key as SA) has 109,684 records and Partition 1 (with key other than SA) has 44,170 records. This tells us that one partition has almost 2.5 times the data as in the other partition. Due to the data skewness we could also see process skewness as the processors process the data. To manage this data skewness we can use a different approach of partitioning like Round robin or random equal  partitioning. This will divide the data equally so that no processor sits idle while other processors are busy doing tasks. ###

## 1.3

- 1

In [70]:
#creating separate rdds for male and female
male = rd_units.filter(lambda x : x[7] in ['Male'] and x[4] != 'Pedestrian on Road')
female = rd_units.filter(lambda x : x[7] in ['Female'] and x[4] != 'Pedestrian on Road')

#getting just the male ages and storing in rdd male_rdd
male_age = male.map(lambda x: x[8])


#getting just the female ages and storing in rdd female_rdd
female_age = female.map(lambda x: x[8])


#function to find avergae of ages of male rdd and female rdd
def find_average(rdd):
    sum = 0
    count = 0
    for val in rdd.collect():
        try:
            sum = sum + int(val)
            if int(val) > 0:
                count = count + 1
        except:
            sum = sum + 0
    return(sum/count)

male_age_avg = find_average(male_age)
female_age_avg = find_average(female_age)

print("Average age of male drivers is - " + str(round(male_age_avg,3)) + "\n" )
print("Average age of female drivers is - " + str(round(female_age_avg,3)))

Average age of male drivers is - 41.0

Average age of female drivers is - 40.386


- 2

In [71]:
#capturing only the years into a separate rdd called year
year = rd_units.map(lambda x: x[5])
#year_final = year.map(lambda x: x.replace("\"",''))

#function to find maximum and minimum year values
def find_maxmin_year(rdd):
    max = 0
    min = 0
    try:
        new_rdd = rdd.filter(lambda x : x.isdigit())
        max = new_rdd.max(key=lambda x: x)
        min = new_rdd.min(key=lambda x: x)
    except:
        pass
    return(max,min)

max_year,min_year = find_maxmin_year(year)

print("Newest vehicle is - " + str(max_year) + "\n")
print("Oldest vehicle is - " + str(min_year))


#finding records with max year
all_max = rd_units.filter(lambda x : x[5] ==  max_year)

#finding records with min year
all_min = rd_units.filter(lambda x : x[5] ==  min_year)


#displaying records with max year
display_max = all_max.map(lambda field: (field[3] ,field[5],field[4]))
print("Records for Newest Vehicle - " + str(display_max.take(3)) + "\n")


#displaying records with min year
display_min = all_min.map(lambda field: (field[3] ,field[5],field[4]))
print("Records for Oldest Vehicle - " + str(display_min.take(3)))



Newest vehicle is - 2019

Oldest vehicle is - 1900
Records for Newest Vehicle - [('SA', '2019', 'Station Wagon'), ('SA', '2019', 'OMNIBUS'), ('SA', '2019', 'Motor Cars - Sedan')]

Records for Oldest Vehicle - [('VIC', '1900', 'Motor Cycle'), ('SA', '1900', 'Motor Cycle'), ('SA', '1900', 'Motor Cycle')]


## 2

- 2.1

- 1

In [72]:
#loading units files to dataframe datarame_units
dataframe_units = spark.read.csv('*_Units.csv',header = True)

#loading crash files to dataframe datarame_crash
dataframe_crash = spark.read.csv('*_Crash.csv',header = True)

- 2

In [73]:
#Displaying schema of the two dataframes
print("Schema of dataframe - dataframe_units")
dataframe_units.printSchema()

print("\n")

print("Schema of dataframe - dataframe_crash")
dataframe_crash.printSchema()

Schema of dataframe - dataframe_units
root
 |-- REPORT_ID: string (nullable = true)
 |-- Unit No: string (nullable = true)
 |-- No Of Cas: string (nullable = true)
 |-- Veh Reg State: string (nullable = true)
 |-- Unit Type: string (nullable = true)
 |-- Veh Year: string (nullable = true)
 |-- Direction Of Travel: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Lic State: string (nullable = true)
 |-- Licence Class: string (nullable = true)
 |-- Licence Type: string (nullable = true)
 |-- Towing: string (nullable = true)
 |-- Unit Movement: string (nullable = true)
 |-- Number Occupants: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Rollover: string (nullable = true)
 |-- Fire: string (nullable = true)



Schema of dataframe - dataframe_crash
root
 |-- REPORT_ID: string (nullable = true)
 |-- Stats Area: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- 

- 2.2

- 1

In [74]:
#Crash events in ADELAIDE where TOTAL CASUALITIES are more than 3

#casting the column Total Cas to int and then filtering greaer than 3 values
cas = dataframe_crash.filter(col("Suburb").isin ('ADELAIDE') & (col("Total Cas").cast("int") > 3))
cas.show()


+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+----------------+--------------+--------------------+------------+-------------+------------+--------+--------------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|Stats Area|  Suburb|Postcode|        LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|     Day|    Time|Area Speed|Position Type|Horizontal Align|Vertical Align|          Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight|    Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+

- 2

In [75]:
#Casting Total Cas to int type and creating a new column CasTmp
dataframe_crash = dataframe_crash.withColumn("CasTmp", dataframe_crash["Total Cas"].cast(IntegerType()))

#Finding top 10 rows with highest casualities by sorting in descending order and displaying top 10 records
highest_cas = dataframe_crash.sort('CasTmp', ascending=False)
highest_cas.show(10)

+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+--------+---------+--------+----------+-------------+--------------------+--------------+--------------+------------+-------------+------------+--------+-----------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+------+
|           REPORT_ID|    Stats Area|         Suburb|Postcode|            LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|      Day|    Time|Area Speed|Position Type|    Horizontal Align|Vertical Align|    Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight| Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|CasTmp|
+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+-

- 3

In [76]:
# Aggregate the dataset by Crash Type and sum the total number of fatalities
agg_attribute1 = 'Crash Type'
sum_attribute = 'Total Fats'

#group by Crash Type and display sum of fatalities in a column 'Total Fatalities'
fats = dataframe_crash.groupby(agg_attribute1).agg(F.sum(sum_attribute).alias('Total Fatalities'))
fats.show()

+--------------------+----------------+
|          Crash Type|Total Fatalities|
+--------------------+----------------+
|           Roll Over|            57.0|
|  Hit Object on Road|             2.0|
|      Hit Pedestrian|            70.0|
|    Hit Fixed Object|           152.0|
|               Other|             2.0|
|          Side Swipe|            20.0|
|             Head On|            86.0|
|  Hit Parked Vehicle|             9.0|
|          Right Turn|            18.0|
|            Rear End|            16.0|
|          Hit Animal|             4.0|
|Left Road - Out o...|             1.0|
|         Right Angle|            45.0|
+--------------------+----------------+



- 4

In [77]:
# Aggregate the dataset by 'Suburb' and sum the total number of casualities

agg_attribute2 = 'Suburb'
sum_attribute = 'Total Cas'

#joining the two dataframes on right join (as we need to fetch records for eaach suburb)
df_units_inner_crash = dataframe_units.join(dataframe_crash,dataframe_units.REPORT_ID==dataframe_crash.REPORT_ID,how='right')

#group by Suburb and display sum of casualities in a column 'Total Casualities'
df_count = df_units_inner_crash.filter(df_units_inner_crash['Licence Type'] == 'Unlicenced')\
                        .groupby(agg_attribute2).agg(F.sum(sum_attribute).alias('Total Casualities'))

df_count.show()

+-------------------+-----------------+
|             Suburb|Total Casualities|
+-------------------+-----------------+
|      FLINDERS PARK|              8.0|
|       POOGINAGORIC|              1.0|
|     TEA TREE GULLY|              1.0|
|            HACKHAM|              3.0|
|   MEDINDIE GARDENS|              0.0|
|           WISANGER|              1.0|
|            CUMMINS|              0.0|
|       BASKET RANGE|              0.0|
|MURRAY BRIDGE SOUTH|              0.0|
|      GILLES PLAINS|              7.0|
|             HAWKER|              0.0|
|           BEAUFORT|              1.0|
|             MAGILL|              7.0|
|            ECHUNGA|              1.0|
|            CULTANA|              1.0|
|        EDWARDSTOWN|              6.0|
|        RISDON PARK|              1.0|
|          THORNGATE|              1.0|
|       ANDREWS FARM|             12.0|
|       TORRENSVILLE|              5.0|
+-------------------+-----------------+
only showing top 20 rows



- 2.3

- 1

In [78]:
# Aggregate the dataset by 'CSEF Severity' and count the total number of records

agg_attribute3 = 'CSEF Severity'

severity = dataframe_crash.groupby(agg_attribute3).agg(F.count(agg_attribute3).alias('Total'))\
                    .orderBy('Total',ascending = False)
severity.show()

+-------------+-----+
|CSEF Severity|Total|
+-------------+-----+
|       1: PDO|46696|
|        2: MI|21881|
|        3: SI| 2978|
|     4: Fatal|  451|
+-------------+-----+



* ### The most common CSEF Severity is of type 1.PDO i.e. Public Damage Only occuring the most times i.e. 46696 times. ###

- 2

- a

In [79]:
# Aggregate the dataset by 'CSEF Severity' and count the total number of records and show its percentage as well

agg_attribute4 =  'CSEF Severity'

#counting the number of records where drugs involved is true
total_drugs = dataframe_crash.filter(dataframe_crash['Drugs Involved'] == 'Y').count()

#Grouping by CSEF Severity and finding count and Percentage of each severity type
positive_drug = dataframe_crash.filter(dataframe_crash['Drugs Involved'] == 'Y')\
                    .groupBy(agg_attribute4) \
                      .count().withColumnRenamed('count', 'Count') \
                        .withColumn('Percentage', sf.round((F.col('Count') / total_drugs) * 100,2) )\
                            .orderBy('Percentage')
  
positive_drug.show()

+-------------+-----+----------+
|CSEF Severity|Count|Percentage|
+-------------+-----+----------+
|     4: Fatal|   82|      6.54|
|       1: PDO|  176|     14.04|
|        3: SI|  247|      19.7|
|        2: MI|  749|     59.73|
+-------------+-----+----------+



- b

In [80]:
# Aggregate the dataset by 'CSEF Severity' and count the total number of records and show its percentage as well

agg_attribute5 =  'CSEF Severity'

#counting the number of records where alcohol involved is true
total_alcohol = dataframe_crash.filter(dataframe_crash['DUI Involved'] == 'Y').count()

#Grouping by CSEF Severity and finding count and Percentage of each severity type
positive_alcohol = dataframe_crash.filter(dataframe_crash['DUI Involved'] == 'Y')\
                    .groupBy(agg_attribute5) \
                      .count().withColumnRenamed('count', 'Count') \
                        .withColumn('Percentage', sf.round((F.col('Count') / total_alcohol) * 100,2) )\
                            .orderBy('Percentage')
  
positive_alcohol.show()

+-------------+-----+----------+
|CSEF Severity|Count|Percentage|
+-------------+-----+----------+
|     4: Fatal|   79|      3.51|
|        3: SI|  259|     11.52|
|        2: MI|  737|     32.78|
|       1: PDO| 1173|     52.18|
+-------------+-----+----------+



- c

In [81]:
# Aggregate the dataset by 'CSEF Severity' and count the total number of records and show its percentage as well

agg_attribute6 =  'CSEF Severity'

#counting the number of records where both drugs and alcohol involved is true
total_drug_alcohol = dataframe_crash.filter((dataframe_crash['DUI Involved'] == 'Y') & (dataframe_crash['Drugs Involved'] == 'Y')).count()


#Grouping by CSEF Severity and finding count and Percentage of each severity type
positive_drugs_alcohol = dataframe_crash.filter((dataframe_crash['DUI Involved'] == 'Y') & (dataframe_crash['Drugs Involved'] == 'Y'))\
                    .groupBy(agg_attribute6) \
                      .count().withColumnRenamed('count', 'Count') \
                        .withColumn('Percentage', sf.round((F.col('Count') / total_drug_alcohol) * 100,2) )\
                            .orderBy('Percentage')

positive_drugs_alcohol.show()
  

+-------------+-----+----------+
|CSEF Severity|Count|Percentage|
+-------------+-----+----------+
|       1: PDO|   24|     13.71|
|     4: Fatal|   27|     15.43|
|        3: SI|   35|      20.0|
|        2: MI|   89|     50.86|
+-------------+-----+----------+



- d

In [82]:
# Aggregate the dataset by 'CSEF Severity' and count the total number of records and show its percentage as well

agg_attribute7 =  'CSEF Severity'

#counting the number of records where both drugs and alcohol involved is false
no_drug_alcohol = dataframe_crash.filter((dataframe_crash['DUI Involved'].isNull() ) & (dataframe_crash['Drugs Involved'].isNull())).count()


#Grouping by CSEF Severity and finding count and Percentage of each severity type
negative_drugs_alcohol = dataframe_crash.filter((dataframe_crash['DUI Involved'].isNull()) & (dataframe_crash['Drugs Involved'].isNull()))\
                    .groupBy(agg_attribute7) \
                      .count().withColumnRenamed('count', 'Count') \
                        .withColumn('Percentage', sf.round((F.col('Count') / no_drug_alcohol) * 100 ,2))\
                            .orderBy('Percentage')


negative_drugs_alcohol.show()
  

+-------------+-----+----------+
|CSEF Severity|Count|Percentage|
+-------------+-----+----------+
|     4: Fatal|  317|      0.46|
|        3: SI| 2507|      3.65|
|        2: MI|20484|     29.83|
|       1: PDO|45371|     66.06|
+-------------+-----+----------+



* ### From the above 4 scenarios we can see that PDO and MI type of severity are seen the most times in terms of percentages. Fatal percentage is usually very less in all the four scenarios when compared to the other types i.e. MI, SI, PDO. However Fatal type is seen more when drugs and alcohol are involved. Also when none of drugs and alcohl are involved then also we can see a high percentage of property damage which is the maximum when comparing all the scenarios.###

- 2.4

- 1 using DATAFRAME

In [83]:
%%time

# 2.4 - 1

# USING DATAFRAME

#joining dataframes on left join to calculate records for each unit
new_join = dataframe_units.join(dataframe_crash,dataframe_units.REPORT_ID==dataframe_crash.REPORT_ID,how='left')


#filtering for suburb ADELAIDE
new_df_crash = new_join.filter(new_join['Suburb'] =='ADELAIDE')\
            .withColumn('Date',sf.concat(sf.col('Year'),sf.lit('-'), sf.col('Month'),sf.lit('-'), sf.col('Day')))\
                .select('Date','Time','Total Cas','Sex','Age', 'Licence Type')

print("Displying records for each unit for Adelaide Suburb")
new_df_crash.show()


Displying records for each unit for Adelaide Suburb
+--------------------+--------+---------+-------+----+------------+
|                Date|    Time|Total Cas|    Sex| Age|Licence Type|
+--------------------+--------+---------+-------+----+------------+
|2016-November-Wed...|01:45 pm|        1|   Male| 056|        Full|
|2016-November-Wed...|01:45 pm|        1|   Male| 072|        null|
|2016-November-Tue...|03:40 pm|        1|   Male| 056|        null|
|2016-November-Tue...|03:40 pm|        1| Female| 027|        null|
|2016-November-Tue...|05:00 pm|        0| Female| 032|        Full|
|2016-November-Tue...|05:00 pm|        0|Unknown| XXX|     Unknown|
|2016-November-Tue...|05:40 pm|        0|   Male| 022|     Unknown|
|2016-November-Tue...|05:40 pm|        0|   Male| 020|     Unknown|
|2016-November-Monday|11:26 pm|        0|Unknown| XXX|     Unknown|
|2016-November-Monday|11:26 pm|        0|   Male| 042|        Full|
|2016-November-Monday|11:26 pm|        0|   null|null|        nu

* 1 using RDD

In [84]:
%%time

# 2.4 - 1

# USING RDD


#function to create Report_ID as key and rest elements as value
def parseNewRecord(line):
    array_line = line[1:]
    # Return a tuple with the REPORT_ID as first element and the remaining as the second element
    return (line[0], array_line)


#creating new rdd for units
rdd_new = rd_units.map(parseNewRecord)

#creating new rdd for crash
rdd2_new = rd_crash.map(parseNewRecord)


#joining the two rdds
joined_rdd = rdd_new.join(rdd2_new)

#filtering data by ADELAIDE
filtered_rdd = joined_rdd.filter(lambda x : x[1][1][1] == 'ADELAIDE')

#Displying the records
filtered_rdd = filtered_rdd.map(lambda field: (field[1][1][9] + '-' +  field[1][1][10] + '-' + field[1][1][11] ,\
                                    field[1][1][12], field[1][1][5],field[1][0][6],field[1][0][7],field[1][0][10]))

#displaying only 10 records
print("Displying records for each unit for Adelaide Suburb")
print(filtered_rdd.take(10))


Displying records for each unit for Adelaide Suburb
[('2016-November-Wednesday', '04:26 pm', '0', 'Male', '017', 'Unknown'), ('2016-November-Wednesday', '04:26 pm', '0', 'Male', '025', 'Unknown'), ('2016-December-Friday', '11:30 am', '0', 'Male', '080', 'Full'), ('2016-December-Friday', '11:30 am', '0', 'Male', '048', 'Full'), ('2016-December-Saturday', '07:40 am', '0', 'Male', '032', 'Full'), ('2016-December-Saturday', '07:40 am', '0', 'Unknown', 'XXX', 'Unknown'), ('2016-December-Friday', '05:30 pm', '0', 'Female', '058', 'Full'), ('2016-December-Friday', '05:30 pm', '0', 'Male', '041', 'Full'), ('2016-December-Wednesday', '04:20 pm', '0', 'Female', '045', 'Full'), ('2016-December-Wednesday', '04:20 pm', '0', 'Male', '027', 'Full')]
CPU times: user 83.7 ms, sys: 13.2 ms, total: 96.9 ms
Wall time: 10.3 s


* 1 using SQL

In [85]:
%%time

# 2.4 - 1

# USING SQL

dataframe_crash.createOrReplaceTempView("sql_crash")
dataframe_units.createOrReplaceTempView("sql_units")

## Join units and crash using SQL
sql_units_left_crash = spark.sql('''
  SELECT CONCAT(c.Year, '-',  c.Month , '-' , c.Day) as Day, c.Time, c.`Total Cas` , u.Sex, u.Age, u.`Licence Type`
  FROM sql_units u LEFT JOIN sql_crash c
  ON c.REPORT_ID = u.REPORT_ID
  WHERE c.Suburb like 'ADELAIDE'
''')

print("Displying records for each unit for Adelaide Suburb")
sql_units_left_crash.show()


Displying records for each unit for Adelaide Suburb
+--------------------+--------+---------+-------+----+------------+
|                 Day|    Time|Total Cas|    Sex| Age|Licence Type|
+--------------------+--------+---------+-------+----+------------+
|2016-November-Wed...|01:45 pm|        1|   Male| 056|        Full|
|2016-November-Wed...|01:45 pm|        1|   Male| 072|        null|
|2016-November-Tue...|03:40 pm|        1|   Male| 056|        null|
|2016-November-Tue...|03:40 pm|        1| Female| 027|        null|
|2016-November-Tue...|05:00 pm|        0| Female| 032|        Full|
|2016-November-Tue...|05:00 pm|        0|Unknown| XXX|     Unknown|
|2016-November-Tue...|05:40 pm|        0|   Male| 022|     Unknown|
|2016-November-Tue...|05:40 pm|        0|   Male| 020|     Unknown|
|2016-November-Monday|11:26 pm|        0|Unknown| XXX|     Unknown|
|2016-November-Monday|11:26 pm|        0|   Male| 042|        Full|
|2016-November-Monday|11:26 pm|        0|   null|null|        nu

* 2 using DATAFRAME

In [86]:
%%time
# 2.4 - 2

# USING DATAFRAME


#joining dataframes on left join to calculate records for each subur
join = dataframe_crash.join(dataframe_units,dataframe_units.REPORT_ID==dataframe_crash.REPORT_ID,how='left')

join2 = join.withColumn("CasTmp", join["Total Cas"].cast(IntegerType()))

# Aggregate the dataset by 'Suburb' and sum the total number of casualities
agg_attribute8 = 'Suburb'
cas_sum = 'CasTmp'

new_df_unlicenced = join2.filter(join2['Licence Type'] =='Unlicenced')\
                        .groupBy(agg_attribute8).agg(F.sum(cas_sum).alias('Total Casualities'))


print("Displying records when Licence Type was Unlicenced")
new_df_unlicenced.show()


    
    

Displying records when Licence Type was Unlicenced
+-------------------+-----------------+
|             Suburb|Total Casualities|
+-------------------+-----------------+
|      FLINDERS PARK|                8|
|       POOGINAGORIC|                1|
|     TEA TREE GULLY|                1|
|            HACKHAM|                3|
|   MEDINDIE GARDENS|                0|
|           WISANGER|                1|
|            CUMMINS|                0|
|       BASKET RANGE|                0|
|MURRAY BRIDGE SOUTH|                0|
|      GILLES PLAINS|                7|
|             HAWKER|                0|
|           BEAUFORT|                1|
|             MAGILL|                7|
|            ECHUNGA|                1|
|            CULTANA|                1|
|        EDWARDSTOWN|                6|
|        RISDON PARK|                1|
|          THORNGATE|                1|
|       ANDREWS FARM|               12|
|       TORRENSVILLE|                5|
+-------------------+--------

- 2 using RDD

In [87]:
%%time

# 2.4 - 2

# USING RDD


#function to create Report_ID as key and rest elements as value
def parseNewRecord(line):
    array_line = line[1:]
    # Return a tuple with the REPORT_ID as first element and the remaining as the second element
    return (line[0], array_line)


#creating new rdd for units
rdd_new = rd_units.map(parseNewRecord)

#creating new rdd for crash
rdd2_new = rd_crash.map(parseNewRecord)



#joining the two rdds
joined_rdd2 = rdd2_new.join(rdd_new)


#filtering data by Unlicenced
joined_rdd2 = joined_rdd2.filter(lambda x : x[1][1][10] == 'Unlicenced')

#Displying the records
new_joined_rdd = joined_rdd2.map(lambda x : (x[1][0][1] , int(x[1][0][5])))

#using mapValues() to get sum of each suburb after grouping by suburb
final_rdd_filtered = new_joined_rdd.groupByKey().mapValues(lambda f : sum(f))

#displaying only 10 records
print("Displying records when Licence Type was Unlicenced")
print(final_rdd_filtered.take(10))

Displying records when Licence Type was Unlicenced
[('ONKAPARINGA HILLS', 7), ('FIRLE', 6), ('RIVERGLADES', 1), ('SALISBURY PARK', 0), ('HAPPY VALLEY', 5), ('MUNNO PARA WEST', 1), ('HEATHPOOL', 3), ('NORTH PLYMPTON', 1), ('WHITES FLAT', 0), ('CRAIGMORE', 2)]
CPU times: user 92 ms, sys: 8.79 ms, total: 101 ms
Wall time: 12.7 s


* 2 using SQL

In [88]:
%%time

# 2.4 - 2
# USING SQL


dataframe_crash.createOrReplaceTempView("sql_crash")
dataframe_units.createOrReplaceTempView("sql_units")


## Join units and crash using SQL
sql_crash_left_units = spark.sql('''
  SELECT c.Suburb , sum(c.`Total Cas`)
  FROM sql_crash c LEFT JOIN sql_units u
  ON c.REPORT_ID = u.REPORT_ID
  WHERE u.`Licence Type` like 'Unlicenced'
  GROUP BY c.Suburb
''')

print("Displying records when Licence Type was Unlicenced")
sql_crash_left_units.show()


Displying records when Licence Type was Unlicenced
+-------------------+------------------------------+
|             Suburb|sum(CAST(Total Cas AS DOUBLE))|
+-------------------+------------------------------+
|      FLINDERS PARK|                           8.0|
|       POOGINAGORIC|                           1.0|
|     TEA TREE GULLY|                           1.0|
|            HACKHAM|                           3.0|
|   MEDINDIE GARDENS|                           0.0|
|           WISANGER|                           1.0|
|            CUMMINS|                           0.0|
|       BASKET RANGE|                           0.0|
|MURRAY BRIDGE SOUTH|                           0.0|
|      GILLES PLAINS|                           7.0|
|             HAWKER|                           0.0|
|           BEAUFORT|                           1.0|
|             MAGILL|                           7.0|
|            ECHUNGA|                           1.0|
|            CULTANA|                           